Цель работы: необходимо для всех юзеров из тестового файла выдать рекомендации и посчитать на actual покупках precision@5 (X5 Retail Hero)

В результате работы над моделью первого уровня были использованы следующие варианты:
    - использование TF-IDF vs BM25 (tf-idf показал результаты выше, чем bm25)
    - использование ItemItemRecommender vs CosineRecommender (CosineRecommender показал результаты выше)
    - использование различных вариантов расчета take_n_popular товаров. В результате исследования данных, лучшим решением 
      было выявлено использование 170 товаров, которые приобретались более 500 раз за отчетный период.

Применение модели второго уровня не улучшило показателей модели первого уровня, не смотря на различное количество получаемых кандидатов, а также добавления различных новых признаков. Было принято решение использовать только модель первого уровня.

Лучший результат метрики precision@5 на исходном датасете показал метод own_recommendations:

    'own_rec' 0.26248256624825517 в сочетании с лучшим показателем precision 0.07084804066780433.

# Import libs

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit import als

# Модель второго уровня
from lightgbm import LGBMClassifier

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

# Написанные функции
from metrics import precision_at_k, recall_at_k
from utils import prefilter_items
from recommenders import MainRecommender

## Read data

In [2]:
PATH_DATA = "data"

In [3]:
data = pd.read_csv(os.path.join(PATH_DATA,'retail_train.csv'))
item_features = pd.read_csv(os.path.join(PATH_DATA,'product.csv'))
user_features = pd.read_csv(os.path.join(PATH_DATA,'hh_demographic.csv'))

# Set global const

In [4]:
ITEM_COL = 'item_id'
USER_COL = 'user_id'
ACTUAL_COL = 'actual'

# N = Neighbors
N_PREDICT = 100

# Process features dataset

In [5]:
# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': ITEM_COL}, inplace=True)
user_features.rename(columns={'household_key': USER_COL }, inplace=True)

# Split dataset for train, eval, test

In [6]:
# Важна схема обучения и валидации!
# -- давние покупки -- | -- 6 недель -- | -- 3 недель -- 
# подобрать размер 2-ого датасета (6 недель) --> learning curve (зависимость метрики recall@k от размера датасета)

VAL_MATCHER_WEEKS = 6
VAL_RANKER_WEEKS = 3

In [7]:
# данные для тренировки matching модели
data_train_matcher = data[data['week_no'] < data['week_no'].max() - (VAL_MATCHER_WEEKS + VAL_RANKER_WEEKS)]

# данные для валидации matching модели
data_val_matcher = data[(data['week_no'] >= data['week_no'].max() - (VAL_MATCHER_WEEKS + VAL_RANKER_WEEKS)) &
                      (data['week_no'] < data['week_no'].max() - (VAL_RANKER_WEEKS))]

# данные для тренировки ranking модели
data_train_ranker = data_val_matcher.copy()  # Для наглядности. Далее добавим изменения, и они будут отличаться

# данные для теста ranking, matching модели
data_val_ranker = data[data['week_no'] >= data['week_no'].max() - VAL_RANKER_WEEKS]

In [8]:
# сделаем объединенный сет данных для первого уровня (матчинга)
df_join_train_matcher = pd.concat([data_train_matcher, data_val_matcher])

In [9]:
def print_stats_data(df_data, name_df):
    print(name_df)
    print(f"Shape: {df_data.shape} Users: {df_data[USER_COL].nunique()} Items: {df_data[ITEM_COL].nunique()}")

In [10]:
print_stats_data(data_train_matcher,'train_matcher')
print_stats_data(data_val_matcher,'val_matcher')
print_stats_data(data_train_ranker,'train_ranker')
print_stats_data(data_val_ranker,'val_ranker')

train_matcher
Shape: (2108779, 12) Users: 2498 Items: 83685
val_matcher
Shape: (169711, 12) Users: 2154 Items: 27649
train_ranker
Shape: (169711, 12) Users: 2154 Items: 27649
val_ranker
Shape: (118314, 12) Users: 2042 Items: 24329


In [11]:
# выше видим разброс по пользователям и товарам и дальше мы перейдем к warm-start (только известные пользователи)

In [12]:
data_val_matcher.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
2104867,2070,40618492260,594,1019940,1,1.00,311,-0.29,40,86,0.0,0.0
2107468,2021,40618753059,594,840361,1,0.99,443,0.00,101,86,0.0,0.0


# Prefilter items

In [13]:
#группировка по количеству для каждого проданного товара
grouped_by_item_id = data_train_matcher.groupby('item_id')['quantity'].agg(['sum']).sort_values(by='sum', ascending=False).head(170)#170 товаров куплены >500 штук
grouped_by_item_id

,sum
item_id,
6534178,175332638
6533889,14518835
6534166,11410640
6544236,2215244
1404121,1468724
...,...
824005,1461
911812,1460
1043590,1460


In [14]:
top_90 = data_train_matcher['quantity'].sum()*0.9
top_90

189835795.8

In [15]:
data_train_matcher.groupby('item_id')['quantity'].agg(['sum']).cumsum()

,sum
item_id,
25671,6
26081,7
26093,8
26190,9
26355,11
...,...
17179426,210928656
17208239,210928657
17208470,210928658


In [16]:
n_items_before = data_train_matcher['item_id'].nunique()

data_train_matcher = prefilter_items(data_train_matcher, item_features=item_features, take_n_popular=170)

n_items_after = data_train_matcher['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

C:\Users\User\Documents\Учеба\GB Data Science\IV четверть\Рекомендательные системы\final project\utils.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['price'] = data['sales_value'] / (np.maximum(data['quantity'], 1))


Decreased # items from 83685 to 171


# Make cold-start to warm-start

In [17]:
# ищем общих пользователей
common_users = data_train_matcher.user_id.values

data_val_matcher = data_val_matcher[data_val_matcher.user_id.isin(common_users)]
data_train_ranker = data_train_ranker[data_train_ranker.user_id.isin(common_users)]
data_val_ranker = data_val_ranker[data_val_ranker.user_id.isin(common_users)]

print_stats_data(data_train_matcher,'train_matcher')
print_stats_data(data_val_matcher,'val_matcher')
print_stats_data(data_train_ranker,'train_ranker')
print_stats_data(data_val_ranker,'val_ranker')

train_matcher
Shape: (861404, 13) Users: 2495 Items: 171
val_matcher
Shape: (169615, 12) Users: 2151 Items: 27644
train_ranker
Shape: (169615, 12) Users: 2151 Items: 27644
val_ranker
Shape: (118282, 12) Users: 2040 Items: 24325


# Init/train recommender

In [18]:
recommender = MainRecommender(data_train_matcher)

### Варианты, как получить кандидатов

Если модель рекомендует < N товаров, то рекомендации дополняются топ-популярными товарами до N

In [19]:
# Берем тестового юзера 2000

In [20]:
recommender.get_als_recommendations(2000, N=5)

[1029743, 8090521, 916122, 874972, 8090537]

In [21]:
recommender.get_own_recommendations(2000, N=5)

[1029743, 8090521, 8090537, 926905, 1050851]

In [22]:
recommender.get_similar_items_recommendation(2000, N=5)

[1122358, 1016800, 1012587, 8090521, 8090509]

In [23]:
recommender.get_similar_users_recommendation(2000, N=5)

[8090521, 1044078, 1029743, 1044078, 1029743]

# Eval recall of matching

### Измеряем recall@k

In [24]:
result_eval_matcher = data_val_matcher.groupby(USER_COL)[ITEM_COL].unique().reset_index()
result_eval_matcher.columns=[USER_COL, ACTUAL_COL]
result_eval_matcher.head(2)

,user_id,actual
0,1,"[853529, 865456, 867607, 872137, 874905, 87524..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870..."


In [25]:
%%time
# для понятности расписано все в строчку, без функций, ваша задача уметь оборачивать все это в функции
result_eval_matcher['own_rec'] = result_eval_matcher[USER_COL].apply(lambda x: recommender.get_own_recommendations(x, N=N_PREDICT))
result_eval_matcher['sim_item_rec'] = result_eval_matcher[USER_COL].apply(lambda x: recommender.get_similar_items_recommendation(x, N=N_PREDICT))
result_eval_matcher['als_rec'] = result_eval_matcher[USER_COL].apply(lambda x: recommender.get_als_recommendations(x, N=N_PREDICT))

Wall time: 6.64 s


In [26]:
%%time
# result_eval_matcher['sim_user_rec'] = result_eval_matcher[USER_COL].apply(lambda x: recommender.get_similar_users_recommendation(x, N=50))

Wall time: 0 ns


### Пример оборачивания

In [27]:
# обернуть в функцию
def evalRecall(df_result, target_col_name, recommend_model):
    result_col_name = 'result'
    df_result[result_col_name] = df_result[target_col_name].apply(lambda x: recommend_model(x, N=25))
    return df_result.apply(lambda row: recall_at_k(row[result_col_name], row[ACTUAL_COL], k=N_PREDICT), axis=1).mean()

In [28]:
# evalRecall(result_eval_matcher, USER_COL, recommender.get_own_recommendations)

In [29]:
def calc_recall(df_data, top_k):
    for col_name in df_data.columns[2:]:
        yield col_name, df_data.apply(lambda row: recall_at_k(row[col_name], row[ACTUAL_COL], k=top_k), axis=1).mean()

In [30]:
def calc_precision(df_data, top_k):
    for col_name in df_data.columns[2:]:
        yield col_name, df_data.apply(lambda row: precision_at_k(row[col_name], row[ACTUAL_COL], k=top_k), axis=1).mean()

### Recall@200 of matching

In [31]:
TOPK_RECALL = 200

In [32]:
sorted(calc_recall(result_eval_matcher, TOPK_RECALL), key=lambda x: x[1],reverse=True)

[('als_rec', 0.07084804066780433),
 ('own_rec', 0.06582125709783601),
 ('sim_item_rec', 0.05827765931303349)]

### Precision@5 of matching

In [33]:
TOPK_PRECISION = 5

In [34]:
sorted(calc_precision(result_eval_matcher, TOPK_PRECISION), key=lambda x: x[1],reverse=True)

[('own_rec', 0.26248256624825517),
 ('als_rec', 0.1692236169223591),
 ('sim_item_rec', 0.06108786610878701)]

### Обучаем модель первого уровня на тестовых данных и сохраняем получившиеся рекомендации

In [35]:
df_test = pd.read_csv('retail_test1.csv')

In [36]:
print_stats_data(df_test,'df_test before')
df_test = df_test[df_test.user_id.isin(common_users)]
print_stats_data(df_test,'df_test after')

df_test before
Shape: (88734, 12) Users: 1885 Items: 20497
df_test after
Shape: (88665, 12) Users: 1883 Items: 20492


In [37]:
result_eval_matcher = df_test.groupby(USER_COL)[ITEM_COL].unique().reset_index()
result_eval_matcher.columns=[USER_COL, ACTUAL_COL]
result_eval_matcher.head(2)

,user_id,actual
0,1,"[880007, 883616, 931136, 938004, 940947, 94726..."
1,2,"[820165, 820291, 826784, 826835, 829009, 85784..."


In [38]:
%%time
# для понятности расписано все в строчку, без функций, ваша задача уметь оборачивать все это в функции
result_eval_matcher['own_rec'] = result_eval_matcher[USER_COL].apply(lambda x: recommender.get_own_recommendations(x, N=5))

Wall time: 1.02 s


In [39]:
print(*sorted(calc_precision(result_eval_matcher, TOPK_PRECISION), key=lambda x: x[1], reverse=True), sep='\n')

('own_rec', 0.18619224641529264)


In [40]:
result = result_eval_matcher[['user_id','own_rec']]
result.to_csv('recommendations.csv', header=None)

# Ranking part

### Обучаем модель 2-ого уровня на выбранных кандидатах

- Обучаем на data_train_ranking
- Обучаем *только* на выбранных кандидатах
- Я *для примера* сгенерирую топ-50 кадидиатов через get_own_recommendations
- (!) Если юзер купил < 50 товаров, то get_own_recommendations дополнит рекоммендации топ-популярными

## Подготовка данных для трейна

In [41]:
# взяли пользователей из трейна для ранжирования
df_match_candidates = pd.DataFrame(data_train_ranker[USER_COL].unique())
df_match_candidates.columns = [USER_COL]

In [42]:
# собираем кандитатов с первого этапа (matcher)
df_match_candidates['candidates'] = df_match_candidates[USER_COL].apply(lambda x: recommender.get_own_recommendations(x, N=N_PREDICT))

In [43]:
df_match_candidates.head(2)

,user_id,candidates
0,2070,"[1029743, 913210, 933067, 838186, 926905, 1016..."
1,2021,"[844179, 1044078, 1013928, 896862, 1000753, 10..."


In [44]:
# разворачиваем товары
df_items = df_match_candidates.apply(lambda x: pd.Series(x['candidates']), axis=1).stack().reset_index(level=1, drop=True)
df_items.name = 'item_id'

In [45]:
df_match_candidates = df_match_candidates.drop('candidates', axis=1).join(df_items)

In [46]:
df_match_candidates.head(4)

,user_id,item_id
0,2070,1029743
0,2070,913210
0,2070,933067
0,2070,838186


### Check warm start

In [47]:
print_stats_data(df_match_candidates, 'match_candidates')

match_candidates
Shape: (215100, 2) Users: 2151 Items: 170


### Создаем трейн сет для ранжирования с учетом кандидатов с этапа 1 

In [48]:
df_ranker_train = data_train_ranker[[USER_COL, ITEM_COL]].copy()
df_ranker_train['target'] = 1  # тут только покупки 

df_ranker_train = df_match_candidates.merge(df_ranker_train, on=[USER_COL, ITEM_COL], how='left')

# чистим дубликаты
df_ranker_train = df_ranker_train.drop_duplicates(subset=[USER_COL, ITEM_COL])

df_ranker_train['target'].fillna(0, inplace= True)

In [49]:
df_ranker_train.target.value_counts()

0.0    178305
1.0      8533
Name: target, dtype: int64

In [50]:
df_ranker_train.head(2)

,user_id,item_id,target
0,2070,1029743,0.0
1,2070,913210,1.0


(!) На каждого юзера 50 item_id-кандидатов

In [51]:
df_ranker_train['target'].mean()

0.04567058093107398

## Подготавливаем фичи для обучения модели

### Описательные фичи

In [52]:
item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [53]:
user_features.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7


In [54]:
df_ranker_train = df_ranker_train.merge(item_features, on='item_id', how='left')
df_ranker_train = df_ranker_train.merge(user_features, on='user_id', how='left')

df_ranker_train.head(2)

,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
0,2070,1029743,0.0,69,GROCERY,Private,FLUID MILK PRODUCTS,FLUID MILK WHITE ONLY,1 GA,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown
1,2070,913210,1.0,2,GROCERY,National,WATER - CARBONATED/FLVRD DRINK,NON-CRBNTD DRNKING/MNERAL WATE,405.6 OZ,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown


**Фичи user_id:**
    - Средний чек
    - Средняя сумма покупки 1 товара в каждой категории
    - Кол-во покупок в каждой категории
    - Частотность покупок раз/месяц
    - Долю покупок в выходные
    - Долю покупок утром/днем/вечером

**Фичи item_id**:
    - Кол-во покупок в неделю
    - Среднее ол-во покупок 1 товара в категории в неделю
    - (Кол-во покупок в неделю) / (Среднее ол-во покупок 1 товара в категории в неделю)
    - Цена (Можно посчитать из retil_train.csv)
    - Цена / Средняя цена товара в категории
    
**Фичи пары user_id - item_id**
    - (Средняя сумма покупки 1 товара в каждой категории (берем категорию item_id)) - (Цена item_id)
    - (Кол-во покупок юзером конкретной категории в неделю) - (Среднее кол-во покупок всеми юзерами конкретной категории в неделю)
    - (Кол-во покупок юзером конкретной категории в неделю) / (Среднее кол-во покупок всеми юзерами конкретной категории в неделю)

### Поведенческие фичи

##### Чтобы считать поведенческие фичи, нужно учесть все данные что были до data_val_ranker

In [55]:
df_join_train_matcher.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [56]:
df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg('sales_value').sum().rename('total_item_sales_value'), how='left',on=ITEM_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg('quantity').sum().rename('total_quantity_value'), how='left',on=ITEM_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg(USER_COL).count().rename('item_freq'), how='left',on=ITEM_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=USER_COL).agg(USER_COL).count().rename('user_freq'), how='left',on=USER_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=USER_COL).agg('sales_value').sum().rename('total_user_sales_value'), how='left',on=USER_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg('quantity').sum().rename('item_quantity_per_week')/df_join_train_matcher.week_no.nunique(), how='left',on=ITEM_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=USER_COL).agg('quantity').sum().rename('user_quantity_per_week')/df_join_train_matcher.week_no.nunique(), how='left',on=USER_COL)


df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg('quantity').sum().rename('item_quantity_per_basket')/df_join_train_matcher.basket_id.nunique(), how='left',on=ITEM_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=USER_COL).agg('quantity').sum().rename('user_quantity_per_baskter')/df_join_train_matcher.basket_id.nunique(), how='left',on=USER_COL)


df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg(USER_COL).count().rename('item_freq_per_basket')/df_join_train_matcher.basket_id.nunique(), how='left',on=ITEM_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=USER_COL).agg(USER_COL).count().rename('user_freq_per_basket')/df_join_train_matcher.basket_id.nunique(), how='left',on=USER_COL)


In [57]:
df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg('coupon_disc').sum().rename('total_item_coupon_disc'), how='left',on=ITEM_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=USER_COL).agg('coupon_disc').sum().rename('total_user_coupon_disc'), how='left',on=USER_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg('coupon_match_disc').sum().rename('total_item_coupon_match_disc'), how='left',on=ITEM_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=USER_COL).agg('coupon_match_disc').sum().rename('total_user_coupon_match_disc'), how='left',on=USER_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg('retail_disc').sum().rename('total_item_retail_disc'), how='left',on=ITEM_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=USER_COL).agg('retail_disc').sum().rename('total_user_retail_disc'), how='left',on=USER_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg('store_id').count().rename('total_item_store_id'), how='left',on=ITEM_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=USER_COL).agg('store_id').count().rename('total_user_store_id'), how='left',on=USER_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=USER_COL).agg('trans_time').min().rename('min_user_trans_time'), how='left',on=USER_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg('trans_time').min().rename('min_item_trans_time'), how='left',on=ITEM_COL)

df_ranker_train = df_ranker_train.merge(((df_join_train_matcher.groupby(by=ITEM_COL).agg('coupon_disc').sum())**2).rename('total_item_coupon_disc**2'), how='left',on=ITEM_COL)

df_ranker_train = df_ranker_train.merge(((df_join_train_matcher.groupby(by=USER_COL).agg('coupon_disc').sum())**2).rename('total_user_coupon_disc**2'), how='left',on=USER_COL)

df_ranker_train = df_ranker_train.merge(((df_join_train_matcher.groupby(by=ITEM_COL).agg('coupon_match_disc').sum())**2).rename('total_item_coupon_match_disc**2'), how='left',on=ITEM_COL)

df_ranker_train = df_ranker_train.merge(((df_join_train_matcher.groupby(by=USER_COL).agg('coupon_match_disc').sum())**2).rename('total_user_coupon_match_disc**2'), how='left',on=USER_COL)

df_ranker_train = df_ranker_train.merge(((df_join_train_matcher.groupby(by=ITEM_COL).agg('retail_disc').sum())**2).rename('total_item_retail_disc**2'), how='left',on=ITEM_COL)

df_ranker_train = df_ranker_train.merge(((df_join_train_matcher.groupby(by=USER_COL).agg('retail_disc').sum())**2).rename('total_user_retail_disc**2'), how='left',on=USER_COL)

df_ranker_train = df_ranker_train.merge(((df_join_train_matcher.groupby(by=ITEM_COL).agg('store_id').count())**2).rename('total_item_store_id**2'), how='left',on=ITEM_COL)

df_ranker_train = df_ranker_train.merge(((df_join_train_matcher.groupby(by=USER_COL).agg('store_id').count())**2).rename('total_user_store_id**2'), how='left',on=USER_COL)

df_ranker_train = df_ranker_train.merge(((df_join_train_matcher.groupby(by=USER_COL).agg('trans_time').min())**2).rename('min_user_trans_time**2'), how='left',on=USER_COL)

df_ranker_train = df_ranker_train.merge(((df_join_train_matcher.groupby(by=ITEM_COL).agg('trans_time').min())**2).rename('min_item_trans_time**2'), how='left',on=ITEM_COL)

df_ranker_train = df_ranker_train.merge(((df_join_train_matcher.groupby(by=ITEM_COL).agg('trans_time').min())**2).rename('min_item_trans_time**2'), how='left',on=ITEM_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=USER_COL).agg('week_no').min().rename('min_user_week_no'), how='left',on=USER_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg('week_no').min().rename('min_item_week_no'), how='left',on=ITEM_COL)

df_ranker_train = df_ranker_train.merge(((df_join_train_matcher.groupby(by=USER_COL).agg('week_no').min())**2).rename('min_user_week_no'), how='left',on=USER_COL)

df_ranker_train = df_ranker_train.merge(((df_join_train_matcher.groupby(by=ITEM_COL).agg('week_no').min())**2).rename('min_item_week_no'), how='left',on=ITEM_COL)

In [58]:
df_ranker_train.head()

,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,...,total_user_retail_disc**2,total_item_store_id**2,total_user_store_id**2,min_user_trans_time**2,min_item_trans_time**2_x,min_item_trans_time**2_y,min_user_week_no_x,min_item_week_no_x,min_user_week_no_y,min_item_week_no_y
0,2070,1029743,0.0,69,GROCERY,Private,FLUID MILK PRODUCTS,FLUID MILK WHITE ONLY,1 GA,45-54,...,1.928960e+06,162231169,3984016,81,0,0,14,1,196,1
1,2070,913210,1.0,2,GROCERY,National,WATER - CARBONATED/FLVRD DRINK,NON-CRBNTD DRNKING/MNERAL WATE,405.6 OZ,45-54,...,1.928960e+06,1380625,3984016,81,256,256,14,2,196,4
2,2070,933067,1.0,1425,MEAT-PCKGD,National,BACON,FLAVORED/OTHER,16 OZ,45-54,...,1.928960e+06,270400,3984016,81,100,100,14,1,196,1
3,2070,838186,1.0,1790,GROCERY,National,BAKED SWEET GOODS,SW GDS:DONUTS,18.2 OZ,45-54,...,1.928960e+06,481636,3984016,81,9,9,14,2,196,4
4,2070,926905,0.0,103,GROCERY,National,SOFT DRINKS,SOFT DRINKS 12/18&15PK CAN CAR,12 OZ,45-54,...,1.928960e+06,305809,3984016,81,1,1,14,1,196,1


In [59]:
X_train = df_ranker_train.drop('target', axis=1)
y_train = df_ranker_train[['target']]

In [60]:
cat_feats = X_train.columns[2:].tolist()
X_train[cat_feats] = X_train[cat_feats].astype('category')

cat_feats

['manufacturer',
 'department',
 'brand',
 'commodity_desc',
 'sub_commodity_desc',
 'curr_size_of_product',
 'age_desc',
 'marital_status_code',
 'income_desc',
 'homeowner_desc',
 'hh_comp_desc',
 'household_size_desc',
 'kid_category_desc',
 'total_item_sales_value',
 'total_quantity_value',
 'item_freq',
 'user_freq',
 'total_user_sales_value',
 'item_quantity_per_week',
 'user_quantity_per_week',
 'item_quantity_per_basket',
 'user_quantity_per_baskter',
 'item_freq_per_basket',
 'user_freq_per_basket',
 'total_item_coupon_disc',
 'total_user_coupon_disc',
 'total_item_coupon_match_disc',
 'total_user_coupon_match_disc',
 'total_item_retail_disc',
 'total_user_retail_disc',
 'total_item_store_id',
 'total_user_store_id',
 'min_user_trans_time',
 'min_item_trans_time',
 'total_item_coupon_disc**2',
 'total_user_coupon_disc**2',
 'total_item_coupon_match_disc**2',
 'total_user_coupon_match_disc**2',
 'total_item_retail_disc**2',
 'total_user_retail_disc**2',
 'total_item_store_id**2

## Обучение модели ранжирования

In [61]:
lgb = LGBMClassifier(objective='binary',
                     max_depth=4,
                     n_estimators=500,
                     learning_rate=0.1,
                     categorical_column=cat_feats)

lgb.fit(X_train, y_train)
cat_feats = X_train.columns[2:].tolist()
train_preds = lgb.predict_proba(X_train)

C:\Users\User\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\User\anaconda3\lib\site-packages\lightgbm\basic.py:863: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


In [62]:
df_ranker_predict = df_ranker_train.copy()

In [63]:
df_ranker_predict['proba_item_purchase'] = train_preds[:,1]

## Подведем итоги

    Мы обучили модель ранжирования на покупках из сета data_train_ranker и на кандитатах от own_recommendations, что является тренировочным сетом, и теперь наша задача предсказать и оценить именно на тестовом сете.

# Evaluation on test dataset

In [64]:
result_eval_ranker = data_val_ranker.groupby(USER_COL)[ITEM_COL].unique().reset_index()
result_eval_ranker.columns=[USER_COL, ACTUAL_COL]
result_eval_ranker.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


## Eval matching on test dataset

In [65]:
%%time
result_eval_ranker['own_rec'] = result_eval_ranker[USER_COL].apply(lambda x: recommender.get_own_recommendations(x, N=N_PREDICT))
#result_eval_ranker['als_rec'] = result_eval_ranker[USER_COL].apply(lambda x: recommender.get_als_recommendations(x, N=N_PREDICT))

Wall time: 1.13 s


In [66]:
# померяем precision только модели матчинга, чтобы понимать влияение ранжирования на метрики

sorted(calc_precision(result_eval_ranker, TOPK_PRECISION), key=lambda x: x[1], reverse=True)

[('own_rec', 0.22705882352941004)]

## Eval re-ranked matched result on test dataset
    Вспомним df_match_candidates сет, который был получен own_recommendations на юзерах, набор пользователей мы фиксировали и он одинаков, значи и прогноз одинаков, поэтому мы можем использовать этот датафрейм для переранжирования.
    

In [67]:
def rerank(user_id):
    return df_ranker_predict[df_ranker_predict[USER_COL]==user_id].sort_values('proba_item_purchase', ascending=False).head(5).item_id.tolist()

In [68]:
result_eval_ranker['reranked_own_rec'] = result_eval_ranker[USER_COL].apply(lambda user_id: rerank(user_id))

## Проверьте данные метрики с фичами и без 

In [69]:
# смотрим на метрики выше и сравниваем что с ранжированием и без, добавляем фичи и то же смотрим
# в первом приближении метрики должны расти с использованием второго этапа

print(*sorted(calc_precision(result_eval_ranker, TOPK_PRECISION), key=lambda x: x[1], reverse=True), sep='\n')

('own_rec', 0.22705882352941004)
('reranked_own_rec', 0.18266318537858764)


C:\Users\User\Documents\Учеба\GB Data Science\IV четверть\Рекомендательные системы\final project\metrics.py:20: RuntimeWarning: invalid value encountered in long_scalars
  return flags.sum() / len(recommended_list)


# Оценка на тесте

In [70]:
df_transactions = pd.read_csv('data/transaction_data.csv')

In [71]:
df_test = pd.read_csv('retail_test1.csv')

In [72]:
print_stats_data(df_test,'df_test before')
df_test = df_test[df_test.user_id.isin(common_users)]
print_stats_data(df_test,'df_test after')

df_test before
Shape: (88734, 12) Users: 1885 Items: 20497
df_test after
Shape: (88665, 12) Users: 1883 Items: 20492


In [73]:
result_test_ranker = df_test.groupby(USER_COL)[ITEM_COL].unique().reset_index()
result_test_ranker.columns=[USER_COL, ACTUAL_COL]
result_test_ranker.head(2)

,user_id,actual
0,1,"[880007, 883616, 931136, 938004, 940947, 94726..."
1,2,"[820165, 820291, 826784, 826835, 829009, 85784..."


In [74]:
%%time
result_test_ranker['own_rec'] = result_test_ranker[USER_COL].apply(lambda x: recommender.get_own_recommendations(x, N=N_PREDICT))
result_test_ranker['reranked_own_rec'] = result_test_ranker[USER_COL].apply(lambda user_id: rerank(user_id))

Wall time: 3.44 s


In [75]:
print(*sorted(calc_precision(result_test_ranker, TOPK_PRECISION), key=lambda x: x[1], reverse=True), sep='\n')

('own_rec', 0.1864046733935189)
('reranked_own_rec', 0.1438063063063055)


C:\Users\User\Documents\Учеба\GB Data Science\IV четверть\Рекомендательные системы\final project\metrics.py:20: RuntimeWarning: invalid value encountered in long_scalars
  return flags.sum() / len(recommended_list)


Вывод - в начале работы.